In [ ]:
# Since we will be using some functions in the numpy, matplotlib, and scipy libraries
# we need to import them to this notebook in order to make use of them
# A good practice is to load them in the first cell of the notebook.
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

# Finding Roots

Often we will come across some complex expression and want to find where it attains a particular value. While the methods and examples shown below are for finding where a function returns zero, you can find where a function $f(x)$ returns an arbitrary value $t$ by finding where $g(x) = f(x) - t = 0$.

## The Bisection Method

This is one of the simplest and most robust methods to find the root of a function, and can be useful when dealing with functions which may not have continuous first derivatives. Convergence typically requires many more iterations than more advanced methods.

Say we have a continuous function and want to find $f(a) = 0$, and we already have $x_0$ and $x_1$ such that $f(x_1)$ and $f(x_0)$ have opposite sign. Then we know there exists $f(a) = 0$ for $x_0 < a < x_1$.

From here, we pick $x_2 = \frac{x_0 + x_1}{2}$ and test the sign of $f(x_2)$. If $f(x_2)$ is of opposite sign to $f(x_0)$, then we set our new interval which contains $a$ to $x_0 < a < x_2$, while if $f(x_2)$ is of opposite sign to $f(x_1)$ our new interval is $x_2 < a < x_1$. Either way, we have halved the interval where we know $a$ exists.

We can repeat this process until we know $a$ to within some arbitrary precision.


### An Example: L-J Potential

As an example for finding the root of a function, let's use the Lennard Jones interatomic potential function:
$$V_{12}(r) = \frac{A}{r^{12}} - \frac{B}{r^6},$$
where $r$ is the interatomic distance and $A$ and $B$ are the coeffiients that describe the short-range repulsive $\frac{1}{r^{12}}$ and long-range attrative $\frac{1}{r^6}$ terms, respectively. Here we will use the parameters for the element Argon:  $A = 1.209\times10^5\ \textrm{eV Angstrom}^{12}$ and $B = 70.66\ \textrm{eV Angstrom}^6$. 

Our intent with this example function is to apply the Bisection method to find the interatomic distance $r$ where the function $V_{12}(r) = 0$, or rather the intercept or root of $V_{12}(r)$. 

Let's first define our function:

In [ ]:
def V(r):
    '''Lennard-Jones Potential for Argon (units are eV with r in Angstrom)'''
    return 1.209E5*r**(-12) - 70.66*r**(-6)


Let's plot the the function to have a look at it. Let's use matplotlib to do this.

In [ ]:
xvals = np.linspace(2, 5, 100)  # 100 points between 2 and 5
plt.plot(xvals, V(xvals))
plt.xlabel('r')
plt.ylabel('V(r)')
plt.grid(False)
plt.show()

# You can not see the features of this function at
# the current scale. Update the above plot to zoom 
# in on the area of the potential function where it
# is near zero. Recall that you can use
# plt.axis([x1, x2, y1, y2]) before the
# plt.show() command to set the axes limits. Additionally
# add grid lines to make it easier to inspect the zero
# intercept of this function. 
# plt.axis([3,4,-0.1,0.2])

In [ ]:
xvals = np.linspace(2, 5, 100)
plt.plot(xvals, V(xvals))
plt.axis([3,4,-0.1,0.2])
plt.show()

By zooming in a little bit we can see there's a root between $r = 3.0$ and $r = 5.0$.

Let's confirm the sign of our function is different at these two values:

In [ ]:
print(V(3.0), V(5.0))
print(1.209E5**(1/12), 70.66**(1/6), (1.209E5/70.66)**(1/6))

Now lets make a function that will use the bisection method to find the value of $r$ to within some tolerance.

In [ ]:
# We'll use a scipy style docstring. This is a good habit to get into
# as it'll let you use "help(function_name)" to get detailed information
# on what the function does, what input it expects, and output it
# generates.

def bisection(func, x0, x1, maxiter, tol):
    '''
    Find the root of a function with the bisection method.

    Parameters
    ----------
    func : function
        A Python function we want to find a root of.
    x0, x1 : float
        The bounding values of the region to find the root.
    maxiter : integer
        The maximum number of iterations.
    tol : float
        Exit when the root is known within this tolerance
    
    Returns
    -------
    x : float
        Root of the function between x0 and x1.
    '''
    f0 = func(x0)
    f1 = func(x1)
    sign_f0 = f0 / abs(f0)
    sign_f1 = f1 / abs(f1)
    # Lets be sure we have valid x0 and x1 before proceeding.
    # We can use assert to raise an error if a condition isn't met.
    assert sign_f0 != sign_f1, "Error: func(x0) and func(x1) do not have opposite sign."
   
    # It's usually a good idea to enforce a limit on how many iterations a loop can
    # do. Otherwise if something goes wrong, it may take some time to understand that
    # your code is not converging.
    for i in range(maxiter):
        x = 0.5 * (x0 + x1)
        # Exit when we know x within tol
        if abs(x-x0) < tol:
            # You can print out some info if you like. This is just for information
            # in this example. You may not want output from functions cluttering a
            # bigger code, and this will interfere with the use of %timeit
            print("bisection converged in", i, "iterations.")
            return x

        fx = func(x)
        sign_fx = fx / abs(fx)
        # Half the range as appropriate before the next iteration
        if sign_fx == sign_f0:
            x0 = x
        else:
            x1 = x

    # If we get to here without returning a value, we haven't converged.
    # "raise" is another good way to get your code to exit with an informative error if
    # an input is not correct. There's a list of exceptions that can be raised at
    # https://docs.python.org/3.6/library/exceptions.html
    raise ValueError("Error: bisection failed to converge after max iterations.")    

In [ ]:
help(bisection)

In [ ]:
%time
bisection(func=V, x0=3.0, x1=5.0, maxiter=100, tol=1e-10)

In [ ]:
# We can use '_' to use the output from the last evaluated cell
# Let's do so to confirm the evaluated value from the bisection function 
# plugged into V gives us zero
V(_)

### Using the Bisection Method with SciPy

Most numerical methods you might want to use have already been implemented as a library or python module. Python modules for potentially intensive calculations tend to be implemented in something that will run more quickly than native python, such as Fortran or C, so it is good to make use of them. Furthermore libraries and modules are likely to be better optimized than code you are likely to write yourself.

SciPy has an implementation of the bisection method in [`scipy.optimize.bisect`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.bisect.html).

In [ ]:
# Uncomment the following for the internal help information on the function
# help(scipy.optimize.bisect)
%time 
scipy.optimize.bisect(f=V, a=3.0, b=5.0, xtol=1e-10)

In [ ]:
V(_)

- Try using the `%timeit` magic to compare the scipy bisection function with the one implemented above. 
    - Be sure to comment out the print statement in the function or `%timeit` will produce a lot of output.

## Newton's Method

This is typically quite a fast converging method, but you need to the know the function analytically. It should have a continuous first derivative, and you need a fairly good initial guess if you want to converge to the desired root.

This works by using a linear expansion of the function using it's derivative at an initial guess to improve upon that guess as shown schematically in the image below:

![Schematic of Newtons Method](images/newton.png)

We can express this mathematically as
$$ x_{i+1} = x_i - \frac{f(x_i)}{f'(x_i)} $$
which we can use to make a function that will iteratively find a root.

Let's make a function that employs Newton's method to we can use to find the root of $\Phi_{12}$ as above.

In [ ]:
def newton(func, dfunc, x0, maxiter, tol):
    '''
    Find the root of a function using Newton's method.

    Parameters
    ----------
    func : function
        A Python function we want to find a root of.
    dfunc : function
        A Python function returning the derivative of func.
    x0 : float
        Initial guess at a root.
    maxiter : integer
        The maximum number of iterations.
    tol : float
        Exit when the root is known within this tolerance

    Returns
    -------
    x : float
        Root of the function.
    '''

    for i in range(maxiter):
        x = x0 - func(x0)/dfunc(x0)
        # Exit when we know x within tol
        if abs(x-x0) < tol:
            # You can print out some info if you like. This is optional, and you may
            # not want output from functions cluttering a bigger code.
            print("newton converged in", i, "iterations.")
            return x
        x0 = x
    
    # If we get to here without returning a value, we haven't converged.
    raise ValueError("Error: newton failed to converge after max iterations.")    

We'll also need to define a function for the derivative:

In [ ]:
def dV(r):
    '''Derivative of Lennard-Jones Potential for Argon (eV with r in Angstrom)'''
    return -12*1.209E5*r**(-13) + 6*70.66*r**(-7)

Now let's find our root, starting from 2.0

In [ ]:
newton(func=V, dfunc=dV, x0=2.0, maxiter=100, tol=1e-10)

As you can see, this took significantly fewer iterations to obtain the same accuracy.

But this method is a bit more sensitive - for example if we take a somewhat higher initial guess we quickly run into trouble:

In [ ]:
newton(func=V, dfunc=dV, x0=4.0, maxiter=100, tol=1e-10)

## Doing this with SciPy

Again, SciPy has an implementation of this we can use as [`scipy.optimize.newton`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html).

In [ ]:
# Uncomment the following for the internal help information on the function
# help(scipy.optimize.newton)

scipy.optimize.newton(func=V, x0=3.0, fprime=dV, tol=1e-10)

## The Secant Method

It can often be tricky to generate a new function that accurately returns the derivative of another function. To get around this we can use the secant method.

This works like Newton's method, but we don't need to have an analytic function for the derivative. Instead we approximate it from two previous guesses as

$$ f'(x_i) \approx \frac{f(x_i) - f(x_{i-1})}{x_i - x_{i-1}} $$

which we can combine with our expression for Newton's method to get

$$ x_{i+1} = x_i - \frac{f(x_i)(x_i - x_{i-1})}{f(x_i) - f(x_{i-1})} = \frac{x_{i-1}f(x_i) - x_{i}f(x_{i-1})}{f(x_i) - f(x_{i-1})}$$

In [ ]:
def secant(func, x0, x1, maxiter, tol):
    '''
    Use the secant method to find a root of a function
    
    Parameters
    ----------
    func : function
        A Python function we want to find a root of.
    x0 : float
        Initial guess at a root.
    x1 : float
        A second guess at a root.
    maxiter : integer
        The maximum number of iterations.
    tol : float
        Exit when the root is known within this tolerance

    Returns
    -------
    x : float
        Root of the function.
    '''

    f0 = func(x0)
    for i in range(maxiter):
        f1 = func(x1)
        x = (x0*f1 - x1*f0)/(f1 - f0)
        # Exit when we know x within tol
        if abs(x-x1) < tol:
            # You can print out some info if you like. This is optional, and you may
            # not want output from functions cluttering a bigger code.
            print("secant converged in", i, "iterations.")
            return x
        x0 = x1
        x1 = x
        f0 = f1

    # If we get to here without returning a value, we haven't converged.
    raise ValueError("Error: secant failed to converge after max iterations.")

In [ ]:
secant(func=V, x0=3.0, x1=3.1, maxiter=100, tol=1e-10)

In [ ]:
# It doesn't matter which of x0 or x1 is a better guess
secant(func=V, x0=3.1, x1=3.0, maxiter=100, tol=1e-10)

In [ ]:
# It is still sensitive to the choice of initial guess though
secant(func=V, x0=3.0, x1=3.9, maxiter=100, tol=1e-10)

### Doing this with SciPy

We can use the same [`scipy.optimize.newton`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html) function as above to use the secant method. This method is used automatically when a derivative function is not provided. Note that the scipy version does not require a second initial guess, it automatically generates the second value by adding a small amount to the provided initial guess.

In [ ]:
scipy.optimize.newton(func=V, x0=3.0, tol=1e-10)

## Brent's Methods

Brent's method uses a combination of ideas from the approaches we discussed above. It decides on each iteration whether it's best to proceed with
- a step of bisection
- a step of secant
- or a step of inverse quadratic interpolation
    - This is like the secant method, but uses results from three values to approximate a quadratic rather than a line.

This gives you the advantages of each method and avoids their drawbacks to provide a robust and fast converging method to obtain a root in a bracketed interval.

It's available in SciPy as [`scipy.optimize.brentq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.brentq.html). **This is likely the best choice of approach available when trying to find a root with Python**.

In [ ]:
scipy.optimize.brentq(f=V, a=3.0, b=5.0, xtol=1e-10, full_output=True)

In [ ]:
%timeit scipy.optimize.brentq(f=V, a=2.0, b=5.0, xtol=1e-10)

In [ ]:
help(scipy.optimize.brentq)